In [36]:
#ECE 189A Capstone P.I.G.S Drone
import socket
import struct
import threading
import time
import random
import json

In [37]:
def get_host_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    finally:
        s.close()
        return ip

In [38]:
PORT_PC = 1145
IP_PHONE = '192.168.127.225'
PORT_PHONE = 1919
IP_DRONE_1 = '192.168.127.18'
PORT_DRONE_1 = 1920
HOST_DRONE_2 = '0.0.0.0'
PORT_DRONE_2 = 1455

#LOCATION FIXED FORMAT
#Source: 0 - Laptop, 1 - Drone 1, 2 - Drone 2
#Latitude: +12.3456789
#Longtitude: +123.4567890
#Overall Format: 0,+12.3456789,+123.4567890
LOCATION = '0,+34.4138888,-119.8488888'

MAX_AMOUNT_TRY = 10

In [39]:
print('Server starts %s' % get_host_ip())

Server starts 169.231.175.129


In [40]:
#!/usr/bin/python
#'''
#Author: Igor Maculan - n3wtron@gmail.com
#A Simple mjpg stream http server
#'''
pcIP = '192.168.0.145'

import cv2
from http.server import BaseHTTPRequestHandler, HTTPServer
from socketserver import ThreadingMixIn

capture = None


class CamHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        if self.path.endswith('.mjpg'):
            self.send_response(200)
            self.send_header(
                'Content-type',
                'multipart/x-mixed-replace; boundary=--jpgboundary'
            )
            self.end_headers()
            while True:
                try:

                    rc, img = capture.read()
                    if not rc:
                        continue

                    img_str = cv2.imencode('.jpg', img)[1].tostring()

                    self.send_header('Content-type', 'image/jpeg')
                    self.send_header('Content-length', len(img_str))
                    self.end_headers()

                    self.wfile.write(img_str)
                    self.wfile.write(b"\r\n--jpgboundary\r\n")

                except KeyboardInterrupt:
                    self.wfile.write(b"\r\n--jpgboundary--\r\n")
                    break
                except BrokenPipeError:
                    continue
            return

        if self.path.endswith('.html'):
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            self.wfile.write(b'<html><head></head><body>')
            self.wfile.write(b'<img src="http://192.168.0.100:8081/cam.mjpg"/>')
            self.wfile.write(b'</body></html>')
            return


class ThreadedHTTPServer(ThreadingMixIn, HTTPServer):
    """Handle requests in a separate thread."""


def main():

    global capture
    capture = cv2.VideoCapture(0)
    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 360)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

    global img
    try:
        server = ThreadedHTTPServer((pcIP, 8081), CamHandler)
        print("server started at http://192.168.0.100:8081/cam.html")
        server.serve_forever()
    except KeyboardInterrupt:
        capture.release()
        server.socket.close()

if __name__ == '__main__':
    main()

OSError: [WinError 10049] The requested address is not valid in its context

In [41]:
#GLOBAL_POSITION_INT {time_boot_ms: 6098353, lat: -353651052, lon: 1491651490, alt: 594070, relative_alt: 10000, vx: -997, vy: -1, vz: 0, hdg: 17993}
# Location: drone ID + lati + longti
# droneID: a b c for drone #1 #2 #3
def drone_location_to_phone(position_str, droneID = "a"):
    #read lati and longti from input
    lati = position_str[position_str.find("lat")+4:position_str.find("lat")+15]
    longti = position_str[position_str.find("lon")+4:position_str.find("lon")+15]
    lati = int(lati) / 10000000
    longti = int(longti) / 10000000
    #print(lati)
    #print(longti)
    return droneID + str(lati) + "," + str(longti)

In [42]:
LOCATION = drone_location_to_phone("GLOBAL_POSITION_INT {time_boot_ms: 6098353, lat: -353651052, lon: 1491651490, alt: 594070, relative_alt: 10000, vx: -997, vy: -1, vz: 0, hdg: 17993}")
print(LOCATION)

da-35.3651052,149.165149


In [43]:
# Location: drone ID + lati + longti
# droneID: da db dc for drone #1 #2 #3
connected = False
def handle_client(client):
    global connected
    
    s1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    s1.connect((IP_DRONE_1,  PORT_DRONE_1))
    print('connected to DRONE1')
    
    request = None
    while True:
        request = client.recv(1024).decode()
        print(request)
        outdata = 'Command:' + request
        request = str(request)
        try:
            s1.send(outdata.encode())
            print('Command send to Drone 1')
        except socket.error:
            s1.close()
            print('Drone 1 lost connection.')
            #break
        client.send(outdata.encode())
        
        if len(request) == 0: # connection closed
            client.close()
            print('client closed connection.')
            break
            
    #client.close()
    connected = False
    #print('client closed connection.')

def send_location(locs, IP, port):
    global connected
    
# Phone
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    #s.bind(("",PORT_DRONE_2))
    print('connected to location update service')
    #print(ipAddr)
    s.connect((IP, port))
    while True:
        #LOCATION = '-' + str(random.randint(-180,180)) + '-' + str(random.randint(-90,90)) + '-'
        try:
            s.send(LOCATION.encode())
        except socket.error:
            s.close()
            print('location server closed connection.')
            break
        #print('Location Sent')
        time.sleep(1)

def start_server():
    global connected
    
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("",PORT_PC))
    server.listen(8)
    print('server start at: %s:%s' % (get_host_ip(), PORT_PC))
    print('wait for connection...')

    try:
        while True:
            client, addr = server.accept()
            connected = True
            print('connected by ' + str(addr))
            threading.Thread(target=handle_client, args=(client,)).start()
    except KeyboardInterrupt:
        server.close()
        print('KeyboardInterrupt')
        

threading.Thread(target=start_server).start() 
threading.Thread(target=send_location, args=(LOCATION, IP_PHONE, PORT_PHONE,)).start() 
threading.Thread(target=send_location, args=(LOCATION, IP_DRONE_1, PORT_PHONE,)).start() 


#while True:
#    LOCATION = '-' + str(random.randint(-180,180)) + '-' + str(random.randint(-90,90)) + '-'
#    print(LOCATION)

connected to location update serviceconnected to location update service
server start at: 169.231.175.129:1145
wait for connection...



Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\huday\anaconda3\envs\CS176C\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\huday\anaconda3\envs\CS176C\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\huday\anaconda3\envs\CS176C\lib\threading.py", line 870, in run
    self.run()
  File "C:\Users\huday\anaconda3\envs\CS176C\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\huday\AppData\Local\Temp\ipykernel_1620\920744506.py", line 44, in send_location
    self._target(*self._args, **self._kwargs)
  File "C:\Users\huday\AppData\Local\Temp\ipykernel_1620\920744506.py", line 44, in send_location
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed

In [ ]:
connected = False
def handle_client(client):
    global connected
    
    s1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    s1.connect((IP_DRONE_1,  PORT_DRONE_1))
    print('connected to DRONE1')
    
    request = None
    while True:
        request = client.recv(1024).decode()
        print(request)
        outdata = 'Command:' + request
        request = str(request)
        try:
            s1.send(outdata.encode())
            print('Command send to Drone 1')
        except socket.error:
            s1.close()
            print('Drone 1 lost connection.')
            #break
        client.send(outdata.encode())
        
        if len(request) == 0: # connection closed
            client.close()
            print('client closed connection.')
            break
            
    #client.close()
    connected = False
    #print('client closed connection.')

def send_location(locs, IP, port):
    global connected
    
# Phone
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    #s.bind(("",PORT_DRONE_2))
    
    #print(ipAddr)
    i = 0
    while (i < MAX_AMOUNT_TRY):
        try:
            s.connect((IP, port))
            print('connected to location update service')
            break
        except:
            time.sleep(3)
            print("Retry Connecting: " + IP)
            i += 1
        
    while True:
        #LOCATION = '-' + str(random.randint(-180,180)) + '-' + str(random.randint(-90,90)) + '-'
        try:
            s.send(LOCATION.encode())
        except socket.error:
            s.close()
            print('location server closed connection.')
            break
        #print('Location Sent')
        time.sleep(1)

def start_server():
    global connected
    
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("",PORT_PC))
    server.listen(8)
    print('server start at: %s:%s' % (get_host_ip(), PORT_PC))
    print('wait for connection...')

    try:
        while True:
            client, addr = server.accept()
            connected = True
            print('connected by ' + str(addr))
            threading.Thread(target=handle_client, args=(client,)).start()
    except KeyboardInterrupt:
        server.close()
        print('KeyboardInterrupt')
        

threading.Thread(target=start_server).start() 
threading.Thread(target=send_location, args=(LOCATION, IP_PHONE, PORT_PHONE,)).start() 
threading.Thread(target=send_location, args=(LOCATION, IP_DRONE_1, PORT_PHONE,)).start() 


while True:
    LOCATION = "0,+" + str(random.randint(0,90)).zfill(2) + '.0000000,+' + str(random.randint(0,180)).zfill(3) +'.0000000'
    #print(LOCATION)

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(("",PORT_PC))
s.listen(5)

#st = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#st.connect(('192.168.127.190', 1145))

print('server start at: %s:%s' % (get_host_ip(), PORT_PC))
print('wait for connection...')
con = 0

while True:
    conn, addr = s.accept()
    print('connected by ' + str(addr))
    DESaddr = addr
    

    while True:
        indata = conn.recv(1024)
        if len(indata) == 0: # connection closed
            conn.close()
            print('client closed connection.')
            break
        print('recv: ' + indata.decode())

        outdata = indata.decode()
        #print(list(DESaddr)[0])
        #print(conn)
        conn.send(LOCATION.encode())
        conn.send(outdata.encode())
        #write_utf8(outdata,conn)
        #st.close()

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(("",PORT_DRONE_2))
s.connect(('192.168.0.4', 1919))

test = "axin223g"
print(test.encode())
cmd = (b'\x00\x014')
s.send(test.encode())
#s.close()

In [ ]:
s.send('Hello!'.encode())

In [ ]:

    while True:
        threading.Timer(10, foo).start()